In [1]:
#These are all libraries that you need to install through the conda navigator for this to run
import cv2
from djitellopy import Tello #Make sure that the file "tello.py" is in the same folder as this file
import time
import os
import cv2
import sys
from enum import Enum
import threading

tello = Tello()
speed = 10
S = 60
FPS = 30

font = cv2.FONT_HERSHEY_PLAIN
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') 
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml') 

#This is a thread that sends the command every 5 seconds.  I think it interferes wth the
#functionality, but I'm not sure. That's why it is commented.
def sendingCommand():
        """
        start a while loop that sends 'command' to tello every 5 second
        #"""    
        while True:
            tello.send_control_command('command')        
            time.sleep(10)

def detect(gray, frame): 
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    for (x, y, w, h) in faces: 
        cv2.rectangle(frame, (x, y), ((x + w), (y + h)), (255, 0, 0), 2) 
        roi_gray = gray[y:y + h, x:x + w] 
        roi_color = frame[y:y + h, x:x + w] 
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 20) 
        eye = eye_cascade.detectMultiScale(roi_gray, 1.8, 20)
  
        for (sx, sy, sw, sh) in smiles: 
            cv2.rectangle(roi_color, (sx, sy), ((sx + sw), (sy + sh)), (0, 0, 255), 2) 
            cv2.putText(frame, "smiling", (50, 50), font, 2,
                    (255, 0, 0), 3)
    return frame 

if not tello.connect():
    print("Tello not connected")
    sys.exit()

if not tello.set_speed(speed):
    print("Not set speed to lowest possible")
    sys.exit()

# In case streaming is on. This happens when we quit this program without the escape key.
if not tello.streamoff():
    print("Could not stop video stream")
    sys.exit()

if not tello.streamon():
    print("Could not start video stream")
    sys.exit()

print ("Current battery is " + tello.get_battery())

#Take off 
tello.takeoff()
time.sleep(6)

# #Move 80cm up
tello.move_up(50)
time.sleep(5)

#Threading. Part of the send command every 10 seconds thing.
#The sending_command will send command to tello every 10 seconds
sending_command_thread = threading.Thread(target = sendingCommand)
sending_command_thread.start()

# def blackDot(gray):
#     # threshold 
#     th, threshed = cv2.threshold(gray, 100, 255, 
#         cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU) 
#     # findcontours 
#     cnts = cv2.findContours(threshed, cv2.RETR_LIST, 
#                     cv2.CHAIN_APPROX_SIMPLE)[-2] 
#     # filter by area 
#     s1 = 3
#     s2 = 20
#     xcnts = [] 
#     for cnt in cnts: 
#         if s1<cv2.contourArea(cnt) <s2: 
#             xcnts.append(cnt)
#     return xcnts

# video_capture = cv2.VideoCapture(0) 
# while True: 
#    # Captures video_capture frame by frame 
#     _, frame = video_capture.read()  
  
#     # To capture image in monochrome                     
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   
      
#     # calls the detect() function     
#     xcnts = blackDot(gray)   
  
#     # Displays the result on camera feed                      
#     cv2.imshow('Video', gray)  
#     cv2.putText(gray, "Dots number: {}".format(len(xcnts)), (50, 50), font, 2, (255,255,255), 3)
#     print("\nDots number: {}".format(len(xcnts))) 
#     # The control breaks once q key is pressed                         
#     if cv2.waitKey(1) & 0xff == ord('q'):                
#         break
  
# # Release the capture once all the processing is done. 
# video_capture.release()                                  
# cv2.destroyAllWindows()
#video_capture = cv2.VideoCapture(0) 
frame_read = tello.get_frame_read()
smile = 0
while True: 
    frame = frame_read.frame
   # Captures video_capture frame by frame 
    #_, frame = video_capture.read()  
  
    # To capture image in monochrome                     
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
#     # Read keybord input from the user
#     if (sys.version_info > (3, 0)):
#       # Python 3 compatibility
#       message = input('')
#     else:
#       # Python 2 compatibility
#       message = raw_input('')
    
#     # If user types quit then lets exit and close the socket
#     if 'quit' in message:
#       print("Program exited sucessfully")
#       sock.close()
    
    # Send the command to Tello
    #tello.send_control_command(message)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    for (x, y, w, h) in faces: 
        cv2.rectangle(frame, (x, y), ((x + w), (y + h)), (255, 0, 0), 2) 
        roi_gray = gray[y:y + h, x:x + w] 
        roi_color = frame[y:y + h, x:x + w] 
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 20) 
        eye = eye_cascade.detectMultiScale(roi_gray, 1.8, 20)
  
        for (sx, sy, sw, sh) in eye: 
            cv2.rectangle(roi_color, (sx, sy), ((sx + sw), (sy + sh)), (0, 0, 255), 2) 
            cv2.putText(frame, "smiling", (50, 50), font, 2,
                    (255, 0, 0), 3)
            #smile = 1
            tello.flip('r')
      
    # calls the detect() function     
    #canvas = detect(gray, frame)    
#     if smile==1:
#         tello.flip('b')
#         sleep(2)
#         smile=0
        
    # Displays the result on camera feed                      
    cv2.imshow('Video', frame)  
  
    # The control breaks once q key is pressed                         
    if cv2.waitKey(1) & 0xff == ord('q'):  
        tello.land();
        break
  
# Release the capture once all the processing is done. 
video_capture.release()                                  
cv2.destroyAllWindows() 

Send command: command
Response: ok
Send command: speed 10
Response: ok
Send command: streamoff
Response: ok
Send command: streamon
Response: ok
Send command: battery?
Response: 100

Current battery is 100

Send command: takeoff
Timeout exceed on command takeoff
Command takeoff was unsuccessful. Message: False
Send command: up 50
Response: ok
Send command: command
Response: ok
Send command: command
Response: ok
Send command: command
Response: ok
Send command: command
Response: ok
Send command: command
Response: ok
Send command: flip r
Response: ok
Send command: command
Response: ok
Send command: flip r
Response: ok
Send command: flip r
Response: error
Command flip r was unsuccessful. Message: error
Send command: flip r
Response: error
Command flip r was unsuccessful. Message: error
Send command: flip r
Response: error
Command flip r was unsuccessful. Message: error
Send command: flip r
Response: error
Command flip r was unsuccessful. Message: error
Send command: flip r
Response: ok
Send

NameError: name 'video_capture' is not defined

Send command: command
Response: ok
Send command: command
Response: ok
Send command: command
Response: ok
Send command: command
Timeout exceed on command command
Command command was unsuccessful. Message: False
